In [5]:
# 기본세팅
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import datetime

# 크롤링용
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import json
import requests

# 컬럼, 로우 조회 최대수 지정 
pd.options.display.max_info_columns =200
pd.options.display.max_columns = 200
pd.options.display.max_info_rows =100
pd.options.display.max_rows = 100

# 시각화할때 warning 무시
import warnings
warnings.filterwarnings(action='ignore')

In [6]:
# 올리브영 가격표기 : 할인가격제외
def replace_price(x):
    for i in range(len(x)):
        x[i] = int(x[i][-10:].replace('원','').replace(',','').replace('~','').replace(' ',''))
    return x

In [7]:
# 올리브영
# 페이지별 정적 크롤링
product_link=[1000001000200010001,1000001000200010006,1000001000200010007,1000001000200010005,1000001000200010009,
              1000001000200010004,1000001000200010002,1000001000200010003,1000001000200010010,1000001000200010005,
              1000001000200070001,1000001000200070005,1000001000200070002,1000001000200070004,1000001000200070003,
              1000001000200060002,1000001000200060005,1000001000200060001,1000001000200060004,1000001000200060003,
              1000001000500030001,1000001000500030002,1000001001000020001,1000001001000020002,1000001001000020003,
              1000001001000020004,1000001000100080001,1000001000100080002,1000001000100080003,1000001000100080004,
              1000001000100090001,1000001000100090002,1000001000100090003,1000001000100100001,1000001000100100002,
              1000001000400100001,1000001000400080001,1000001000400080004,1000001000400080005,1000001000400080002,
              1000001001100010001,1000001001100010002]
product_name=['BB/CC','블러셔/치크','쉐어딩/컨투어링','컨실러','쿠션',
              '파우더/팩트','파운데이션','프라이머/베이스','픽서','하이라이트',
              '마스카라','메이크업세트','아이라이너','아이브로우','아이섀도우/팔레트',
              '립글로스','립라이너','립밤','립스틱','립틴트',
              '향수','향수기타','클렌징오일/밤','클렌징워터','립&아이리무버',
              '클렌징밀크/크림','스킨/토너','로션/에멀전','올인원','스킨케어 세터',
              '에센스/세럼','크림','아이크림','미스트/픽서','페이스오일',
              '컬러염색','샴푸','샴푸바','드라이샴푸','린스/컨디셔너',
              '선크림','선스틱/선스프레이']

# 크롤링 일시에 따른 파일명 설정을 위한 date 변수 설정
date=str(datetime.date.today()).replace('-','')[2:]

# 전체 데이터
olive_df=pd.DataFrame()
for category in product_link:

    # 카테고리 정보
    cat_df=pd.DataFrame()
    for pagenum in range(1,5):
        url = f'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo={category}&fltDispCatNo=&prdSort=01&pageIdx={pagenum}&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat{category}_Small'

        seed = np.random.randint(100)
        np.random.seed(seed)    
        a = np.random.randint(5)
        time.sleep(a)
        try:

            response = requests.get(url)   
        except: 
            print('현재 카테고리 연결실패')
            break
            
        data = BeautifulSoup(response.text, 'html.parser')
        
        # 현재 페이지 정보
        brand = [a.text for a in data.select('span.tx_brand')]
        product = [b.text for b in data.select('p.tx_name')]
        price = [c.text for c in data.select('p.prd_price')]
        replace_price(price)
        if not brand:
            break
        
        else:
        # 카테고리 정보에 현재 페이지 정보 넣기
            page_df = pd.DataFrame(columns=['brand','product','price'], data=list(zip(brand,product,price)))
            temp = page_df
            cat_df = pd.concat([cat_df,temp])
            cat_df['category'] = product_name[product_link.index(category)]
    print(f'{product_name[product_link.index(category)]} 인기상품 크롤링 완료\n')
                 
    # 전체 데이터에 카테고리 정보 넣기
    temp_df=cat_df
    olive_df=pd.concat([olive_df,temp_df])
    if category != 1000001001100010002:
        print('*****다음 카테고리 작업중******')
    else:
        print('*****올리브영 크롤링 완료******\n')
        
olive_df = olive_df[['category','brand','product','price']]
olive_df.index = range(len(olive_df))

# 데이터저장
olive_df.to_csv(f'/Users/ppangppang/Documents/dev/TIL/test_beauty_data/{date}_olive_best.csv', index=False)
# 백업
olive_df.to_csv(f'/Users/ppangppang/Desktop/ssac/data_crwaling/beauty_test/databackup/{date}_olive_best.csv', index=False)
print('*****올리브영 인기상품 크롤링 완료******')

BB/CC 인기상품 크롤링 완료

*****다음 카테고리 작업중******
블러셔/치크 인기상품 크롤링 완료

*****다음 카테고리 작업중******
쉐어딩/컨투어링 인기상품 크롤링 완료

*****다음 카테고리 작업중******
컨실러 인기상품 크롤링 완료

*****다음 카테고리 작업중******
쿠션 인기상품 크롤링 완료

*****다음 카테고리 작업중******
파우더/팩트 인기상품 크롤링 완료

*****다음 카테고리 작업중******
파운데이션 인기상품 크롤링 완료

*****다음 카테고리 작업중******
프라이머/베이스 인기상품 크롤링 완료

*****다음 카테고리 작업중******
픽서 인기상품 크롤링 완료

*****다음 카테고리 작업중******
컨실러 인기상품 크롤링 완료

*****다음 카테고리 작업중******
마스카라 인기상품 크롤링 완료

*****다음 카테고리 작업중******
메이크업세트 인기상품 크롤링 완료

*****다음 카테고리 작업중******
아이라이너 인기상품 크롤링 완료

*****다음 카테고리 작업중******
아이브로우 인기상품 크롤링 완료

*****다음 카테고리 작업중******
아이섀도우/팔레트 인기상품 크롤링 완료

*****다음 카테고리 작업중******
립글로스 인기상품 크롤링 완료

*****다음 카테고리 작업중******
립라이너 인기상품 크롤링 완료

*****다음 카테고리 작업중******
립밤 인기상품 크롤링 완료

*****다음 카테고리 작업중******
립스틱 인기상품 크롤링 완료

*****다음 카테고리 작업중******
립틴트 인기상품 크롤링 완료

*****다음 카테고리 작업중******
향수 인기상품 크롤링 완료

*****다음 카테고리 작업중******
향수기타 인기상품 크롤링 완료

*****다음 카테고리 작업중******
클렌징오일/밤 인기상품 크롤링 완료

*****다음 카테고리 작업중******
클렌징워터 인기상품 크롤링 완료

*****다음 카테고리 작업중******
립

In [8]:
olive_df

,category,brand,product,price
0,BB/CC,셀퓨전씨,셀퓨전씨 스킨 블레미쉬 밤 인텐시브 40ml,25600
1,BB/CC,아이디얼포맨,아이디얼포맨 블레미시커버비비크림(보통피부용),16720
2,BB/CC,닥터자르트,닥터자르트 더메이크업 디스어포어 뷰티 밤_비비크림 50ml,24000
3,BB/CC,헤라,헤라 옴므 CC크림 SPF35/PA++ 50ml(내추럴베이지/다크베이지),34000
4,BB/CC,아이디얼포맨,아이디얼포맨 다크커버톤업크림,16720
...,...,...,...,...
2689,선스틱/선스프레이,온더바디,온더바디 워터프루프 투명 선스틱,17900
2690,선스틱/선스프레이,비오템 옴므,비오템 옴므 UV 디펜스 선스틱 20g,42000
2691,선스틱/선스프레이,비오템 옴므,비오템 옴므 UV 디펜스 선스틱 20g 세트,42000
2692,선스틱/선스프레이,온더바디,온더바디 내츄럴 산뜻 파우더리 선스틱,17900
